

# Evaluate gas species properties

PyChemkin inherits many Ansys Chemkin utilities for evaluating species properties in a mechanism.
Most tools for calculating the species properties are part of the PyChemkin ``Chemistry()`` methods.
This example shows how use some of these methods to evaluate species thermodynamic and transport properties.


## Import PyChemkin packages and start the logger



In [ ]:
import os

import ansys.chemkin as ck  # Chemkin
from ansys.chemkin.logger import logger
import matplotlib.pyplot as plt  # plotting
import numpy as np  # number crunching

# check working directory
current_dir = os.getcwd()
logger.debug("working directory: " + current_dir)
# set verbose mode
ck.set_verbose(True)
# set interactive mode for plotting the results
# interactive = True: display plot
# interactive = False: save plot as a PNG file
global interactive
interactive = True

## Create a chemistry set
The mechanism to load is the GRI 3.0 mechanism for methane combustion.
This mechanism and its associated data files come with the standard Ansys Chemkin
installation in the ``/reaction/data`` directory.



In [ ]:
# set mechanism directory (the default Chemkin mechanism data directory)
data_dir = os.path.join(ck.ansys_dir, "reaction", "data")
mechanism_dir = data_dir

# create a chemistry set based on GRI 3.0
MyGasMech = ck.Chemistry(label="GRI 3.0")
# set mechanism input files
# inclusding the full file path is recommended
MyGasMech.chemfile = os.path.join(mechanism_dir, "grimech30_chem.inp")
MyGasMech.thermfile = os.path.join(mechanism_dir, "grimech30_thermo.dat")
MyGasMech.tranfile = os.path.join(mechanism_dir, "grimech30_transport.dat")

## Preprocess the chemistry set



In [ ]:
# preprocess the mechanism files
iError = MyGasMech.preprocess()

## Display the basic mechanism information
Display the element and species information from the GRI 3.0 mechanism. You can get the entire list
of the elements and the gas species in the mechanism by using the ``element_symbols`` and
``species_symbols`` lists, respectively.

Use the ``get_specindex()`` method to get the species index of a species in the mechanism. Use
the ``SpeciesComposition()`` method to check the elemental composition of a gas species.

<div class="alert alert-info"><h4>Note</h4><p>Both the species name/symbol and element name/symbol are case-sensitive.</p></div>



In [ ]:
# extract element symbols as a list
elelist = MyGasMech.element_symbols
# extract gas species symbols as a list
specieslist = MyGasMech.species_symbols
# list of gas species of interest
plotspeclist = ["CH4", "O2", "N2"]
# find elemental compositions of selected species
print(" ")
for s in plotspeclist:
    speciesID = MyGasMech.get_specindex(s)
    print("species " + specieslist[speciesID])
    print("elemental composition")
    for elemID in range(MyGasMech.MM):
        num_elem = MyGasMech.SpeciesComposition(elemID, speciesID)
        print(f"    {elelist[elemID]:>4}: {num_elem:2d}")
    print("=" * 10)
print()

## Plot selected species properties
Calculate and plot the properties of CH\ :sub:`4`\ , O\ :sub:`2`\ , and N\ :sub:`2` against the
temperature. Use these property methods:

- ``SpeciesCv``: Species specific heat capacity at constant volume [erg/mol-K]
- ``SpeciesCond``: Species thermal conductivity [erg/cm-K-sec]
- ``SpeciesDiffusionCoeffs``: Binary diffusion coefficients between species pairs [cm\ :sup:`2`\ /sec]




In [ ]:
# plot Cv and thermal conductivity values at different temperatures for selected gas species
#
plt.figure(figsize=(12, 6))
# temperature increment
dTemp = 20.0
# number of property data points
points = 100
# curve attributes
curvelist = ["g", "b--", "r:"]

## Prepare the species Cv data for plotting



In [ ]:
# create arrays
# specify specific heat capacity at constant volume data
Cv = np.zeros(points, dtype=np.double)
# temperature data
T = np.zeros(points, dtype=np.double)
# start plotting loop #1
k = 0
# loop over selected gas species
for s in plotspeclist:
    # starting temperature at 300 [K]
    Temp = 300.0
    # loop over temperature data points
    for i in range(points):
        HeatCapacity = MyGasMech.SpeciesCv(Temp)
        ID = MyGasMech.get_specindex(s)
        T[i] = Temp
        # convert ergs to joules
        Cv[i] = HeatCapacity[ID] / ck.ergs_per_joule
        Temp += dTemp
    plt.subplot(121)
    plt.plot(T, Cv, curvelist[k])
    k += 1
# plot Cv versus temperature
plt.xlabel("Temperature [K]")
plt.ylabel("Cv [J/mol-K]")
plt.legend(plotspeclist, loc="upper left")

## Prepare the species thermal conductivity data for plotting



In [ ]:
# create arrays
# specify conductivity
kappa = np.zeros(points, dtype=np.double)
# start plotting loop #2
k = 0
# loop over selected gas species
for s in plotspeclist:
    # starting temperature at 300 [K]
    Temp = 300.0
    # loop over temperature data points
    for i in range(points):
        conductivity = MyGasMech.SpeciesCond(Temp)
        ID = MyGasMech.get_specindex(s)
        T[i] = Temp
        # convert ergs to joules
        kappa[i] = conductivity[ID] / ck.ergs_per_joule
        Temp += dTemp
    plt.subplot(122)
    plt.plot(T, kappa, curvelist[k])
    k += 1
# plot conductivity versus temperature
plt.xlabel("Temperature [K]")
plt.ylabel("Conductivity [J/cm-K-sec]")
plt.legend(plotspeclist, loc="upper left")

## Evaluate the binary diffusion coefficients between different gas species
Use the ``SpeciesDiffusionCoeffs()`` method to calculate the binary diffusion coefficients
between pairs of gas species. Here, the binary diffusion coefficients are evaluated at 2 [atm]
and 500 [K]. The binary diffusion coefficient between CH\ :sub:`4` and O\ :sub:`2` is shown.



In [ ]:
diffcoef = MyGasMech.SpeciesDiffusionCoeffs(2.0 * ck.Patm, 500.0)
ID1 = MyGasMech.get_specindex(plotspeclist[0])
ID2 = MyGasMech.get_specindex(plotspeclist[1])
c = diffcoef[ID1][ID2]
print(
    f"Diffusion coefficient for {plotspeclist[0]} against {plotspeclist[1]} is {c:e} [cm²/sec]."
)

# plot results
if interactive:
    plt.show()
else:
    plt.savefig("plot_species_properties.png", bbox_inches="tight")